## 📚 Quick demo notebook – Bayesian Optimization with risk of non-convergence

This short notebook shows how to combine  
* **PyTorch** – tensor backend  
* **GPyTorch** – Gaussian-process models  
* **BoTorch** – Bayesian-optimization utilities  
* **Weights & Biases** – experiment tracking  

### What happens?

1. **Two surrogate models** are created  
   * `gp_reward`  – *regression* GP for the objective value  
   * `gp_conv`    – *classification* GP (Bernoulli likelihood) for the probability that a run converges  

2. They are wrapped in a **`ModelListGP`** container so we can query a joint posterior in a single call.

3. We build an **acquisition function**  

A(x) = ExpectedImprovement_reward(x) × P_convergence(x)

This favors points that promise a high improvement **and** have a decent chance to finish successfully.

4. `optimize_acqf` searches the acquisition landscape and proposes `q = 3` new hyper-parameter configurations.

5. For each proposed config we launch *k* replicas, log `(converged?, reward)`, and update the surrogates – the classic BO loop.

> **Goal of the notebook:** verify that the two-head ModelList works,
> inspect EI × Pconv visually, and ensure the code runs before plugging in your real training loop.


In [2]:
import torch
import gpytorch
from botorch.models import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.optim import optimize_acqf
import wandb


In [16]:
import botorch
from botorch.models.transforms.outcome import Logit

ImportError: cannot import name 'Logit' from 'botorch.models.transforms.outcome' (C:\Users\Admin\AppData\Roaming\Python\Python313\site-packages\botorch\models\transforms\outcome.py)

In [5]:
# Données binaires (0 / 1).  eps évite logit(0) ou logit(1)
X        = torch.rand(40, 2)
Y_raw    = (torch.rand(40) < 0.7).float().unsqueeze(-1)  # (40 × 1)
eps      = 1e-3
Y_prob   = Y_raw * (1 - 2*eps) + eps                     # ∈ (0,1)

In [21]:
gp_conv = SingleTaskGP(
    train_X = X,
    train_Y = Y_prob,
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2328\2690184970.py:1: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  gp_conv = SingleTaskGP(


In [22]:
gp_reward = SingleTaskGP(
    train_X = X,
    train_Y = Y_prob,
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2328\2727844786.py:1: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  gp_reward = SingleTaskGP(


In [23]:
model = ModelListGP(gp_reward, gp_conv)